In [1]:
import boto3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'omar-vargas-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target 
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [3]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3349.000000,3375.000000,3389.000000
mean,49.580236,0.496165,236.937892,25.836290,75.753615
std,8.592057,0.500059,45.319255,4.115615,11.905116
min,32.000000,0.000000,107.000000,15.960000,45.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,234.000000,25.410000,75.000000
75%,56.000000,1.000000,263.000000,28.060000,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [4]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,839.000000,844.000000,848.000000
mean,49.603774,0.485849,235.858164,25.664917,76.379717
std,8.497160,0.500095,41.568044,3.934325,12.494992
min,33.000000,0.000000,129.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,22.987500,68.000000
50%,49.000000,0.000000,235.000000,25.345000,75.000000
75%,56.000000,1.000000,261.500000,27.940000,82.000000
max,68.000000,1.000000,372.000000,45.790000,122.000000


In [5]:
## Defining the imputer
imputer = KNNImputer(n_neighbors = 5,  weights = 'distance').fit(X_train)

## Imputing thr missing values
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [6]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.580236,0.496165,236.906720,25.840003,75.753368
std,8.592057,0.500059,45.150363,4.112010,11.903368
min,32.000000,0.000000,107.000000,15.960000,45.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,234.000000,25.410000,75.000000
75%,56.000000,1.000000,263.000000,28.060000,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [7]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.603774,0.485849,235.944989,25.657923,76.379717
std,8.497160,0.500095,41.411730,3.927564,12.494992
min,33.000000,0.000000,129.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,22.987500,68.000000
50%,49.000000,0.000000,235.000000,25.340000,75.000000
75%,56.000000,1.000000,261.250000,27.940000,82.000000
max,68.000000,1.000000,372.000000,45.790000,122.000000


Random Forest

In [8]:
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

## Predicting on the test
RF_pred = RF_md.predict_proba(X_test_imp)[:,1]

## Changing likelihood to labels
RF_label = np.where(RF_pred < 0.1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.92      0.39      0.55       719
           1       0.19      0.81      0.31       129

    accuracy                           0.45       848
   macro avg       0.56      0.60      0.43       848
weighted avg       0.81      0.45      0.51       848



AdaBoost

In [10]:
Ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 5), n_estimators = 500, learning_rate = 0.01).fit(X_train_imp, Y_train)

## Predicting on the test
Ada_pred = Ada_md.predict_proba(X_test_imp)[:,1]

## Changing likelihood to labels
Ada_label = np.where(Ada_pred < 0.1, 0, 1)

print(classification_report(Y_test, Ada_label))

              precision    recall  f1-score   support

           0       0.88      0.16      0.27       719
           1       0.16      0.88      0.27       129

    accuracy                           0.27       848
   macro avg       0.52      0.52      0.27       848
weighted avg       0.77      0.27      0.27       848



In [ ]:
## Based on my results (using recall) I would use the Random Forest model to predict TenYearCHD.